In [1]:
import pandas as pd

In [2]:
path_faire = '/Users/luke.thompson/Google Drive/My Drive/NOAA/mbon-seus/metadata/FAIRe-ODE_noaa-aoml-seusmbon_20250909.xlsx'
output_directory = '.' #'/Users/luke.thompson/Google Drive/My Drive/NOAA/mbon-seus/metadata/metadata'

In [3]:
dict_assay2short = {'Bacteria-16S-V4V5-Parada': '16S', 
                    'Eukarya-18S-V9-AmaralZettler': '18S'}


In [4]:
df_project = pd.read_excel(path_faire, sheet_name='projectMetadata', comment='#')
df_sample = pd.read_excel(path_faire, sheet_name='sampleMetadata', comment='#')
df_exptrun = pd.read_excel(path_faire, sheet_name='experimentRunMetadata', comment='#')

In [5]:
# Remove first 2 columns from df_project, set field_name column as the index, and transpose the DataFrame
df_project_first_2_cols_removed = df_project.iloc[:, 2:]
df_project_first_2_cols_removed.set_index('term_name', inplace=True)
df_project2 = df_project_first_2_cols_removed.transpose()
# Fill missing values in rows after first row using the values from first row, and avoid downcasting warning
first_row_values = df_project2.loc['project_level']
df_project3 = df_project2.fillna(first_row_values)

/var/folders/rk/h_1wslx16sxbwsx5k1s2jn2h0000gn/T/ipykernel_54060/3044486737.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_project3 = df_project2.fillna(first_row_values)


In [6]:
df_project2

term_name,recordedBy,recordedByID,project_contact,institution,institutionID,project_name,project_id,parent_project_id,study_factor,assay_type,...,lib_layout,adapter_forward,adapter_reverse,lib_screen,checksum_method,seq_method_additional,expedition_id,ship_crs_expocode,woce_sect,bioproject_accession
project_level,Luke Thompson | Sammy Harding,https://orcid.org/0000-0002-3911-1280 | https:...,luke.thompson@noaa.gov,NOAA Atlantic Oceanographic and Meteorological...,https://ror.org/042r9xb17,Southeast US MBON,noaa-aoml-seusmbon,NaN,water column spatial series,metabarcoding,...,paired end,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bacteria-16S-V4V5-Parada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,ACACTGACGACATGGTTCTACA,TACGGTAGCAGAGACTTGGTCT,The Genomics Core performed secondary PCR usin...,MD5,PhiX control library was spiked in at 10%.,NaN,NaN,NaN,NaN
Eukarya-18S-V9-AmaralZettler,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,ACACTGACGACATGGTTCTACA,TACGGTAGCAGAGACTTGGTCT,The Genomics Core performed secondary PCR usin...,MD5,PhiX control library was spiked in at 10%.,NaN,NaN,NaN,NaN


In [7]:
df_project3

term_name,recordedBy,recordedByID,project_contact,institution,institutionID,project_name,project_id,parent_project_id,study_factor,assay_type,...,lib_layout,adapter_forward,adapter_reverse,lib_screen,checksum_method,seq_method_additional,expedition_id,ship_crs_expocode,woce_sect,bioproject_accession
project_level,Luke Thompson | Sammy Harding,https://orcid.org/0000-0002-3911-1280 | https:...,luke.thompson@noaa.gov,NOAA Atlantic Oceanographic and Meteorological...,https://ror.org/042r9xb17,Southeast US MBON,noaa-aoml-seusmbon,NaN,water column spatial series,metabarcoding,...,paired end,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bacteria-16S-V4V5-Parada,Luke Thompson | Sammy Harding,https://orcid.org/0000-0002-3911-1280 | https:...,luke.thompson@noaa.gov,NOAA Atlantic Oceanographic and Meteorological...,https://ror.org/042r9xb17,Southeast US MBON,noaa-aoml-seusmbon,NaN,water column spatial series,metabarcoding,...,paired end,ACACTGACGACATGGTTCTACA,TACGGTAGCAGAGACTTGGTCT,The Genomics Core performed secondary PCR usin...,MD5,PhiX control library was spiked in at 10%.,NaN,NaN,NaN,NaN
Eukarya-18S-V9-AmaralZettler,Luke Thompson | Sammy Harding,https://orcid.org/0000-0002-3911-1280 | https:...,luke.thompson@noaa.gov,NOAA Atlantic Oceanographic and Meteorological...,https://ror.org/042r9xb17,Southeast US MBON,noaa-aoml-seusmbon,NaN,water column spatial series,metabarcoding,...,paired end,ACACTGACGACATGGTTCTACA,TACGGTAGCAGAGACTTGGTCT,The Genomics Core performed secondary PCR usin...,MD5,PhiX control library was spiked in at 10%.,NaN,NaN,NaN,NaN


In [8]:
# From df_project2, get the project_id
project_id = df_project2['project_id'].iloc[0]
project_id

'noaa-aoml-seusmbon'

In [9]:
# From df_exptrun get the unique values of the 'seq_run_id' and 'assay_name' columns
seq_run_ids = df_exptrun['seq_run_id'].unique()
assay_names = df_exptrun['assay_name'].unique()

In [10]:
# Check if the values of 'assay_name' are the same in df_project and df_exptrun
set(assay_names) == set(df_exptrun['assay_name'].dropna())

True

Add code to generate manifests also.

In [11]:
# for each assay_name, get the corresponding rows of df_exptrun, merge with df_samples on samp_name, add project/assay-specific metadata as new columns, drop empty columns, and save to a tsv file
for assay_name in df_exptrun['assay_name'].unique():
    df = df_exptrun[df_exptrun['assay_name'] == assay_name].merge(df_sample, on='samp_name', how='left')
    df = pd.merge(df, df_project3.loc[assay_name].to_frame().transpose(), how='cross')    
    df.dropna(axis=1, how='all', inplace=True)
    df.to_csv(f"{output_directory}/seusmbon_{dict_assay2short[assay_name]}_metadata.tsv", sep='\t', index=False)

In [12]:
df

,samp_name,assay_name_x,pcr_plate_id,lib_id,seq_run_id,filename,filename2,checksum_filename,checksum_filename2,input_read_count,...,sequencing_location,platform_y,instrument,seq_kit,lib_layout,adapter_forward,adapter_reverse,lib_screen,checksum_method,seq_method_additional
0,SEUSMBON_WS22022_MR_MR_Bot_C,Eukarya-18S-V9-AmaralZettler,mbon_18s_p7_r1,WS22022-2C-18S-20231113,20231113_AND14518_Amplicon_PE150,WS22022-2C-18S_S1_L001_R1_001.fastq.gz,WS22022-2C-18S_S1_L001_R2_001.fastq.gz,bb4e709d53b7fcb43662c9f2362ab65f,fac04264a1b6af64eaf0ba49602b0a7d,13519,...,Michigan State University Research Technology ...,ILLUMINA,Illumina MiSeq [OBI_0002003],MiSeq v2 500,paired end,ACACTGACGACATGGTTCTACA,TACGGTAGCAGAGACTTGGTCT,The Genomics Core performed secondary PCR usin...,MD5,PhiX control library was spiked in at 10%.
1,SEUSMBON_WS22022_LK_LK_Bot_B,Eukarya-18S-V9-AmaralZettler,mbon_18s_p7_r1,WS22022-6B-18S-20231113,20231113_AND14518_Amplicon_PE150,WS22022-6B-18S_S2_L001_R1_001.fastq.gz,WS22022-6B-18S_S2_L001_R2_001.fastq.gz,4951bd62fa8ddc653bf22b26224d6d05,a7276cbf198776a9a85ae92ee7732e64,13314,...,Michigan State University Research Technology ...,ILLUMINA,Illumina MiSeq [OBI_0002003],MiSeq v2 500,paired end,ACACTGACGACATGGTTCTACA,TACGGTAGCAGAGACTTGGTCT,The Genomics Core performed secondary PCR usin...,MD5,PhiX control library was spiked in at 10%.
2,SEUSMBON_WS22022_MR_MR_Sur_C,Eukarya-18S-V9-AmaralZettler,mbon_18s_p7_r1,WS22022-1C-18S-20231113,20231113_AND14518_Amplicon_PE150,WS22022-1C-18S_S3_L001_R1_001.fastq.gz,WS22022-1C-18S_S3_L001_R2_001.fastq.gz,9be9aa46840bcae078e16d53521e2841,e29c95d03fb23f6ac9553f6e31359d6d,7453,...,Michigan State University Research Technology ...,ILLUMINA,Illumina MiSeq [OBI_0002003],MiSeq v2 500,paired end,ACACTGACGACATGGTTCTACA,TACGGTAGCAGAGACTTGGTCT,The Genomics Core performed secondary PCR usin...,MD5,PhiX control library was spiked in at 10%.
3,SEUSMBON_WS22022_KWN_KW1_Sur_C,Eukarya-18S-V9-AmaralZettler,mbon_18s_p7_r1,WS22022-10C-18S-20231113,20231113_AND14518_Amplicon_PE150,WS22022-10C-18S_S4_L001_R1_001.fastq.gz,WS22022-10C-18S_S4_L001_R2_001.fastq.gz,664708203e3a885880123516b5a1a21e,53bf933ed1cfd8b8e0dafa13bdbc6248,10076,...,Michigan State University Research Technology ...,ILLUMINA,Illumina MiSeq [OBI_0002003],MiSeq v2 500,paired end,ACACTGACGACATGGTTCTACA,TACGGTAGCAGAGACTTGGTCT,The Genomics Core performed secondary PCR usin...,MD5,PhiX control library was spiked in at 10%.
4,SEUSMBON_WS22022_MI_30_Sur_B,Eukarya-18S-V9-AmaralZettler,mbon_18s_p7_r1,WS22022-12B-18S-20231113,20231113_AND14518_Amplicon_PE150,WS22022-12B-18S_S5_L001_R1_001.fastq.gz,WS22022-12B-18S_S5_L001_R2_001.fastq.gz,41205e0d0747bac9d7298d43893a92c1,b4e191fb9eac70e2c941ab8a08b2610d,10006,...,Michigan State University Research Technology ...,ILLUMINA,Illumina MiSeq [OBI_0002003],MiSeq v2 500,paired end,ACACTGACGACATGGTTCTACA,TACGGTAGCAGAGACTTGGTCT,The Genomics Core performed secondary PCR usin...,MD5,PhiX control library was spiked in at 10%.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3925,SEUSMBON_ZymoMock_Plate9_AVITI,Eukarya-18S-V9-AmaralZettler,mbon_18s_p9_r1,ZymoMock-SEMBONp9-18S-20250808,20250808_THO16657_18S-Amplicon_AVITI_PE150,ZymoMock-SEMBONp9-18S_L2_R1.fastq.gz,ZymoMock-SEMBONp9-18S_L2_R2.fastq.gz,c23652c0943fc3c3a158769799056227,117f10f074aeb6484c19b567f6af7bd3,379818,...,Michigan State University Research Technology ...,ILLUMINA,Illumina MiSeq [OBI_0002003],MiSeq v2 500,paired end,ACACTGACGACATGGTTCTACA,TACGGTAGCAGAGACTTGGTCT,The Genomics Core performed secondary PCR usin...,MD5,PhiX control library was spiked in at 10%.
3926,SEUSMBON_RTSF_NTC_1322544_Plate39_18S,Eukarya-18S-V9-AmaralZettler,not applicable: control sample,RTSF_NTC_1322544-20250808,20250808_THO16657_18S-Amplicon_AVITI_PE150,RTSF_NTC_1322544_L2_R1.fastq.gz,RTSF_NTC_1322544_L2_R2.fastq.gz,f0d79988b7772c003d04a28bd7417a62,f0d79988b7772c003d04a28bd7417a62,0,...,Michigan State University Research Technology ...,ILLUMINA

In [13]:
# Create dict of DataFrames to store the merged DataFrames
merged_md = {}
merged_man = {}

# Loop through each unique combination of 'seq_run_id' and 'assay_name'
for seq_run_id in seq_run_ids:
    for assay in assay_names:
        # Get the subset of df_exptrun for the current 'seq_run_id' and 'assay_name'
        df_exptrun_subset = df_exptrun[(df_exptrun['seq_run_id'] == seq_run_id) & (df_exptrun['assay_name'] == assay)]
        # Merge the subset of df_exptrun with df_sample using 'samp_name' as the key and add to list of merged DataFrames
        metadata = pd.merge(df_exptrun_subset, df_sample, left_on='samp_name', right_on='samp_name')
        # If the merged DataFrame is not empty, continue
        if metadata.shape[0] > 0:
            # Create a dictionary to hold the new columns
            new_columns = {}
            # Add each column from df_project2 to the dictionary
            for column in df_project2.columns:
                new_columns[column] = df_project2.loc['project_level', column]
                if not pd.isna(df_project2.loc[assay, column]):
                    new_columns[column] = df_project2.loc[assay, column]
            # Convert the dictionary to a DataFrame with the same indexes as the merged DataFrame, repeating the values for each row
            new_columns_df = pd.DataFrame(new_columns, index=metadata.index)
            # Concatenate the new columns DataFrame with the merged DataFrame
            metadata = pd.concat([metadata, new_columns_df], axis=1)
            # Drop columns that contain only NaN values
            metadata = metadata.dropna(axis=1, how='all')
            # Add merged DataFrame to the dict of merged DataFrames
            merged_md[(seq_run_id, assay)] = metadata
            # Save the merged DataFrame to a tab-delimited file and drop the index column
            metadata.to_csv(f'metadata.{project_id}.{seq_run_id}.{assay}.tsv', sep='\t', index=False)